In [1]:
import os
import time
import datetime
import re
import shutil
from collections import deque
import argparse

import numpy as np
import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from simulation.simulations.data_generator import DataGenerator
from TransformerMOT.util.misc import save_checkpoint, update_logs
from TransformerMOT.util.load_config_files import load_yaml_into_dotdict
from TransformerMOT.util.plotting import output_truth_plot, compute_avg_certainty, get_constrastive_ax, get_false_ax, \
    get_total_loss_ax, get_state_uncertainties_ax
from TransformerMOT.util.logger import Logger
from TransformerMOT.models.BOMTv1 import BOMT
from simulation.simulations.data_generator import DataGenerator, get_single_training_example


In [2]:
task_params = r"C:\Users\chiny\OneDrive - Nanyang Technological University\Y3S2 (Internship)\MultiTracking\configs\tasks\task2.yaml"
model_params = r"C:\Users\chiny\OneDrive - Nanyang Technological University\Y3S2 (Internship)\MultiTracking\configs\models\BOMTv1.yaml"

params = load_yaml_into_dotdict(task_params)
params.update(load_yaml_into_dotdict(model_params))

if params.general.pytorch_and_numpy_seed is None:
    random_data = os.urandom(4)
    params.general.pytorch_and_numpy_seed = int.from_bytes(random_data, byteorder="big")
print(f'Using seed: {params.general.pytorch_and_numpy_seed}')

if params.training.device == 'auto':
    params.training.device = 'cuda' if torch.cuda.is_available() else 'cpu'

Using seed: 3084929566


In [3]:
data_generator = DataGenerator(params=params)
training_nested_tensor, labels, unique_measurement_ids = data_generator.get_batch()
training_nested_tensor.tensors.shape

torch.Size([5, 329, 4])

In [4]:
model1 = BOMT(params).to("cuda")
checkpoint = torch.load(r"results\train2_removed_sigmoid\checkpoints\checkpoint_gradient_step_513.pth",map_location="cuda")
model1.load_state_dict(checkpoint["model_state_dict"])
res = model1(training_nested_tensor.to("cuda"))

measurements:  {torch.Size([5, 329, 2])}
embeddings:  {torch.Size([329, 5, 256])}
normalized_measurements_presigmoid: torch.Size([5, 329, 2])
memory_padding_mask: torch.Size([5, 329])


In [5]:
for i in res[1]:
    print(i.logits.shape)

torch.Size([5, 16, 1])
torch.Size([5, 16, 1])
torch.Size([5, 16, 1])
torch.Size([5, 16, 1])
torch.Size([5, 16, 1])


In [6]:
res[0].positions

tensor([[[0.1107, 0.2603],
         [0.1596, 0.2268],
         [0.1091, 0.1851],
         ...,
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000]],

        [[0.0505, 0.0172],
         [0.1367, 0.1295],
         [0.0985, 0.2281],
         ...,
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000]],

        [[0.0478, 0.0414],
         [0.0773, 0.1279],
         [0.1184, 0.2046],
         ...,
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000]],

        [[0.0993, 0.2090],
         [0.1237, 0.0855],
         [0.1481, 0.2538],
         ...,
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000]],

        [[0.1286, 0.2391],
         [0.1770, 0.1404],
         [0.1408, 0.1386],
         ...,
         [0.1069, 0.1729],
         [0.1255, 0.1220],
         [0.0633, 0.1053]]], device='cuda:0', grad_fn=<SliceBackward0>)